In [1]:
import sys
if "/content/drive/My Drive/snakeAI" not in sys.path:
  sys.path.append("/content/drive/My Drive/snakeAI")

import tensorflow as tf
import tensorflow.keras as k

from Snake.environment import SnakeMaze

from Snake.utils import resize_image, save_images, save_video,save_eval
from Snake.utils import save_graph, generate_animation, euclidean_distance
from Snake.agent import Snake

from Snake.variables import Cell, Status
from Model.replay_memory import ReplayMemory, Experience
from Model.model_train import train_dqn

import matplotlib.pyplot as plt

import numpy as np
import os

from time import time
from datetime import datetime

In [2]:
def get_observation(env, handle=0, local_knowledge=5):
  snake = env.snakes[handle]

  if snake.status == Status.DEAD:
    return np.zeros((33 + local_knowledge**2 ))

  snake_position = snake.body[0]

  env_width, env_height, = env.matrix.shape

  directions_offsets = [(0, 1), (-1, 0), (1, 0), (0, -1), (-1, 1), (1, -1), (1, 1), (-1, -1)]
  features = []

  for offset_x, offset_y in directions_offsets: 
    distances_to_cell_type = {c: -1 for c in Cell.CELL_DICT.values()}
    tmp_x, tmp_y = snake_position
    tmp_x += offset_x
    tmp_y += offset_y
    
    distance = 1
    while (-1 < tmp_x < env_height) and (-1 < tmp_y < env_width):
      curr_cell_type = env.matrix[tmp_x, tmp_y]
      if distances_to_cell_type[curr_cell_type] == -1:
        distances_to_cell_type[curr_cell_type] = distance
      distance += 1
      tmp_x += offset_x
      tmp_y += offset_y

    del distances_to_cell_type[Cell.EMPTY_CELL]
    features.extend(distances_to_cell_type.values())
  
  features.append(len(snake.body))

  tmp_matrix = np.pad(env.matrix, local_knowledge)
  tmp_matrix = tmp_matrix[
                          snake_position[0]: snake_position[0]+local_knowledge, 
                          snake_position[1]: snake_position[1]+local_knowledge, 
                          ]
  features.extend(tmp_matrix.flatten())
  
  return np.array(features)  

def reward(snake: Snake, env: SnakeMaze, direction):
    if snake.status == Status.DEAD:
        return -500
    else:
        r = 0
        r += snake.steps_without_food * (-2)
        r += 350 if snake.steps_without_food == 1 else 0
        return r

In [3]:
NUM_NOT_TRAINABLE_BLOCKS = 4

IMAGE_SIZE = (64, 64)
IMAGE_SHAPE = (*IMAGE_SIZE, 3)

learning_rate = 1e-5

config = {
  "max_steps_per_episode": 500,
  "num_rolling_avg_sample": 10,
  "evaluate_each": 100,
  "save_models": True,
  "save_graphs": True,
  "gamma": .9,
  "epsilon": 0,
  "epsilon_decay": 0.00005,
  "boundaries": True,
  "maze_width": 10,
  "batch_size": 128,
  "get_observation": get_observation,
  "path_to_weights": os.path.join(*['drive', 'My Drive', 'snakeAI', 'trainings', "NonConvolutional", "29Jun2020__174338130864", "model"]),
  "comment": f"""
  Architecture: Non-convolutional
  Optimizer: Adam ( with amsgrad )
  Hyperparameters:
    lr = {learning_rate}
  Reward:
    if snake.status == Status.DEAD:
        return -500
    else:
        r = 0
        r += snake.steps_without_food * (-2)
        r += 350 if snake.steps_without_food == 1 else 0
        return r
  """
}

In [ ]:
  model = k.Sequential([
                      k.layers.Input(58, name='InputLayer'),
                      k.layers.Dense(40, activation='relu', kernel_regularizer=k.regularizers.l1_l2(), name='FirstDense'),
                      k.layers.Dropout(.5, name='FirstDropout'),
                      k.layers.Dense(20, activation='relu', kernel_regularizer=k.regularizers.l1_l2(), name='SecondDense'),
                      k.layers.Dropout(.5, name='SecondDropout'),
                      k.layers.Dense(4, activation='softmax', kernel_regularizer=k.regularizers.l1_l2(), name='ThirdDense'),
], name='NonSequential')
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, amsgrad=True)
config['training_dir'] = os.path.join(*['drive', 'My Drive', 'snakeAI', 'trainings', "NonConvolutional", datetime.now().strftime('%d%h%Y__%H%M%S%f')])
train_dqn(model, optimizer, reward, **config)

Streaming output truncated to the last 5000 lines.
____________________________________________________________________________________________
Episode 1290 Done!
Episode reward:           -494
Epsilon:                  0.01
Number of random actions: [0]
Replay Memory size: 244

____________________________________________________________________________________________
Episode 1291 Done!
Episode reward:           -494
Epsilon:                  0.01
Number of random actions: [0]
Replay Memory size: 248

____________________________________________________________________________________________
Episode 1292 Done!
Episode reward:           -486
Epsilon:                  0.01
Number of random actions: [1]
Replay Memory size: 256

____________________________________________________________________________________________
Episode 1293 Done!
Episode reward:           -494
Epsilon:                  0.01
Number of random actions: [0]
Replay Memory size: 260

_________________________________

/content/drive/My Drive/snakeAI/Snake/utils.py:56: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1, ax2) = plt.subplots(2, 1)


Graph saved at: drive/My Drive/snakeAI/trainings/NonConvolutional/29Jun2020__203653913460/Reward.png
____________________________________________________________________________________________
Episode 2002 Done!
Episode reward:           -486
Epsilon:                  0.01
Number of random actions: [0]
Replay Memory size: 312

____________________________________________________________________________________________
Episode 2003 Done!
Episode reward:           32
Epsilon:                  0.01
Number of random actions: [4]
Replay Memory size: 323

____________________________________________________________________________________________
Episode 2004 Done!
Episode reward:           -239238
Epsilon:                  0.01
Number of random actions: [3]
Replay Memory size: 151

____________________________________________________________________________________________
Episode 2005 Done!
Episode reward:           -486
Epsilon:                  0.01
Number of random actions: [0]
Replay 

/content/drive/My Drive/snakeAI/Snake/utils.py:56: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1, ax2) = plt.subplots(2, 1)


Graph saved at: drive/My Drive/snakeAI/trainings/NonConvolutional/29Jun2020__203653913460/Reward.png
____________________________________________________________________________________________
Episode 2102 Done!
Episode reward:           -486
Epsilon:                  0.01
Number of random actions: [0]
Replay Memory size: 250

____________________________________________________________________________________________
Episode 2103 Done!
Episode reward:           -484
Epsilon:                  0.01
Number of random actions: [0]
Replay Memory size: 259

____________________________________________________________________________________________
Episode 2104 Done!
Episode reward:           -18388
Epsilon:                  0.01
Number of random actions: [4]
Replay Memory size: 375

____________________________________________________________________________________________
Episode 2105 Done!
Episode reward:           1000
Epsilon:                  0.01
Number of random actions: [5]
Replay

...

Graph saved at: drive/My Drive/snakeAI/trainings/NonConvolutional/29Jun2020__203653913460/Reward.png
____________________________________________________________________________________________
Episode 4702 Done!
Episode reward:           338
Epsilon:                  0.01
Number of random actions: [0]
Replay Memory size: 377

____________________________________________________________________________________________
Episode 4703 Done!
Episode reward:           -200
Epsilon:                  0.01
Number of random actions: [1]
Replay Memory size: 343

____________________________________________________________________________________________
Episode 4704 Done!
Episode reward:           -472
Epsilon:                  0.01
Number of random actions: [0]
Replay Memory size: 358

____________________________________________________________________________________________
Episode 4705 Done!
Episode reward:           -400
Epsilon:                  0.01
Number of random actions: [1]
Replay Me

/content/drive/My Drive/snakeAI/Snake/utils.py:56: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1, ax2) = plt.subplots(2, 1)


Graph saved at: drive/My Drive/snakeAI/trainings/NonConvolutional/29Jun2020__203653913460/Reward.png
____________________________________________________________________________________________
Episode 4802 Done!
Episode reward:           -252
Epsilon:                  0.01
Number of random actions: [0]
Replay Memory size: 285

____________________________________________________________________________________________
Episode 4803 Done!
Episode reward:           -492
Epsilon:                  0.01
Number of random actions: [0]
Replay Memory size: 290

____________________________________________________________________________________________
Episode 4804 Done!
Episode reward:           -492
Epsilon:                  0.01
Number of random actions: [0]
Replay Memory size: 295

____________________________________________________________________________________________
Episode 4805 Done!
Episode reward:           -488
Epsilon:                  0.01
Number of random actions: [0]
Replay M

/content/drive/My Drive/snakeAI/Snake/utils.py:56: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1, ax2) = plt.subplots(2, 1)


Graph saved at: drive/My Drive/snakeAI/trainings/NonConvolutional/29Jun2020__203653913460/Reward.png
____________________________________________________________________________________________
Episode 4902 Done!
Episode reward:           -480
Epsilon:                  0.01
Number of random actions: [0]
Replay Memory size: 299

____________________________________________________________________________________________
Episode 4903 Done!
Episode reward:           -452
Epsilon:                  0.01
Number of random actions: [0]
Replay Memory size: 324

____________________________________________________________________________________________
Episode 4904 Done!
Episode reward:           -434
Epsilon:                  0.01
Number of random actions: [1]
Replay Memory size: 326

____________________________________________________________________________________________
Episode 4905 Done!
Episode reward:           -488
Epsilon:                  0.01
Number of random actions: [0]
Replay M